In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [2]:
import csv
import json
import pandas as pd
import numpy as np
from datetime import datetime
import re

## Speeches URLs¶

In [7]:
web = "https://www.presidency.ucsb.edu"
campaignroot0 = "/documents/app-categories/elections-and-transitions/convention-speeches"
campaignrootpage = requests.get(web + campaignroot0)

In [8]:
campaignrootsoup = BeautifulSoup(campaignrootpage.text, "html.parser")

In [9]:
campaignroot = []
campaignroot.append(campaignroot0)
bottom = campaignrootsoup.findAll('ul', attrs = {'class':'pagination'})[0].findAll('a')

while bottom[-2].text == 'next ›':
    nexturl = bottom[-2]['href']
    campaignroot.append(nexturl)
    page = requests.get(web + nexturl)
    soup = BeautifulSoup(page.text,"html.parser")
    bottom = soup.findAll('ul', attrs = {'class':'pagination'})[0].findAll('a')

In [11]:
TextURLs = []
for lk in campaignroot:
    pg = requests.get(web + lk)
    soup = BeautifulSoup(pg.text, "html.parser")
    table = soup.findAll('div', attrs = {'class':'field-title'})
    for tb in table:
        TextURLs.append(tb.a['href'])
    time.sleep(0.05)

## Scrape Speeches

In [12]:
pg = requests.get(web + TextURLs[0])
soup = BeautifulSoup(pg.text, "html.parser")
Content = soup.find('div', attrs = {'class':'field-docs-content'}).text

In [15]:
for i,url in enumerate(TextURLs):
    pg = requests.get(web + url)
    soup = BeautifulSoup(pg.text, "html.parser")
    Title = soup.find('div', attrs={'class':'field-ds-doc-title'}).text
    Time = soup.find('div', attrs = {'class':'field-docs-start-date-time'}).text
    Content = soup.find('div', attrs = {'class':'field-docs-content'}).text
    Name = soup.find('h3', attrs={'class':'diet-title'}).text
    Type = "Convention Speeches"
    time.sleep(0.05)
    if i in list(range(0,3000,100)):
        print(i)
    with open('ConventionSpeech.csv', 'a') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow([Name, Title, Time, Content, Type, url])

0


## Merging with speech data

In [8]:
Speech = pd.read_csv('CampaignSpeech.csv')

In [9]:
Speech.tail()

,Candidate,Title,Date,Content,Type,URL,Year,Month,Day,Speech_index
2734,Franklin D. Roosevelt,"\nAddress at Oglethorpe University in Atlanta,...","\nMay 22, 1932","\nPresident Jacobs, members and friends of Ogl...",Campaign Documents,/documents/address-oglethorpe-university-atlan...,1932,May,22,Speech2734
2735,Franklin D. Roosevelt,"\nAddress at Jefferson Day Dinner in St. Paul,...","\nApril 18, 1932",\nWhat is the real reason that Jefferson Day D...,Campaign Documents,/documents/address-jefferson-day-dinner-st-pau...,1932,April,18,Speech2735
2736,Franklin D. Roosevelt,"\nRadio Address From Albany, New York: ""The 'F...","\nApril 07, 1932",\nAlthough I understand that I am talking unde...,Campaign Documents,/documents/radio-address-from-albany-new-york-...,1932,April,7,Speech2736
2737,Franklin D. Roosevelt,\nThe Governor Enters the First Primary Campai...,"\nJanuary 22, 1932",\nMy dear Mr. McLean:\nIf it is the desire of ...,Campaign Documents,/documents/the-governor-enters-the-first-prima...,1932,January,22,Speech2737
2738,William Howard Taft,\nRemarks Accepting the Presidential Nominatio...,"\nAugust 01, 1912",\nMR. ROOT AND CHAIRMEN OF THE NOTIFICATION CO...,Campaign Documents,/documents/remarks-accepting-the-presidential-...,1912,August,1,Speech2738


In [4]:
ConventionSpeech = pd.read_csv('ConventionSpeech.csv', header=None)

In [5]:
ConventionSpeech.columns = ['Candidate','Title','Date','Content','Type','URL']
ConventionSpeech.head()

,Candidate,Title,Date,Content,Type,URL
0,Hillary Clinton,\nAddress Accepting the Presidential Nominatio...,"\nJuly 28, 2016","\nThank you all very, very much! Thank you for...",Convention Speeches,/documents/address-accepting-the-presidential-...
1,Donald J. Trump,\nAddress Accepting the Presidential Nominatio...,"\nJuly 21, 2016","\nFriends, delegates and fellow Americans: I h...",Convention Speeches,/documents/address-accepting-the-presidential-...
2,Barack Obama,\nRemarks Accepting the Presidential Nominatio...,"\nSeptember 06, 2012",\nThe First Lady. Thank you so much. Tonight I...,Convention Speeches,/documents/remarks-accepting-the-presidential-...
3,Mitt Romney,\nAddress Accepting the Presidential Nominatio...,"\nAugust 30, 2012","\nMr. Chairman, delegates. I accept your nomin...",Convention Speeches,/documents/address-accepting-the-presidential-...
4,John McCain,\nAddress Accepting the Presidential Nominatio...,"\nSeptember 04, 2008",\nThank you. Thank you all very much. Thank yo...,Convention Speeches,/documents/address-accepting-the-presidential-...


In [6]:
ConventionSpeech['Year'] = [int(re.split(',',x)[1].strip()) for x in ConventionSpeech.Date]
ConventionSpeech['Month'] = [re.split(',',x)[0].strip('\n')[:-3] for x in ConventionSpeech.Date]
ConventionSpeech['Day'] = [int(re.split(',',x)[0].strip('\n')[-2:]) for x in ConventionSpeech.Date]

In [7]:
ConventionSpeech = ConventionSpeech[(ConventionSpeech.Year < 1952) | (ConventionSpeech.Year > 1996)]

In [10]:
ConventionSpeech['Speech_index']= ['Speech'+ str(x) for x in range(len(Speech.Title), len(Speech.Title) + len(ConventionSpeech.Title))]

In [12]:
len(ConventionSpeech.Speech_index)

25

In [17]:
Speech = Speech.append(ConventionSpeech)

In [295]:
Speech.to_csv("CampaignSpeech2.csv", index = False)

In [299]:
ConventionSpeech.to_csv("ConventionSpeech2.csv", index=False)

## Merge at Subspeech Level

In [14]:
sub_dict = {'Subspeech_index':[], 'SubContent':[], 'Speech_index':[]}

In [13]:
ConventionSpeech = pd.read_csv("ConventionSpeech2.csv")

In [15]:
# Save every 10 lines into a subspeech e.g. [0,10)

for i,text in enumerate(ConventionSpeech.Content): 
    splited = re.split('\n',text)
    length = len(re.split('\n',text))
    if length > 10:
        a = list(range(0,length,10)) 
        for j in a[:-2]:
            rowvalue = []
            upper=j+10
            sub_dict['Subspeech_index'].append(ConventionSpeech['Speech_index'][i]+'_'+str(j)+ ':' + str(upper))
            sub_dict['SubContent'].append(splited[j: upper])
            sub_dict['Speech_index'].append(ConventionSpeech['Speech_index'][i])
            
        sub_dict['Subspeech_index'].append(ConventionSpeech['Speech_index'][i]+'_'+ str(a[-2])+ ':' + str(length))
        sub_dict['SubContent'].append(splited[a[-2]: length])
        sub_dict['Speech_index'].append(ConventionSpeech['Speech_index'][i])
    else:
        sub_dict['Subspeech_index'].append(ConventionSpeech['Speech_index'][i] +'_'+str(0)+ ':' + str(length))
        sub_dict['SubContent'].append(splited[: length])
        sub_dict['Speech_index'].append(ConventionSpeech['Speech_index'][i])

In [16]:
Sub_Speech = pd.DataFrame.from_dict(sub_dict)
len(Sub_Speech.Speech_index.unique())

25

In [18]:
SubSpeech = Sub_Speech.merge(ConventionSpeech, left_on='Speech_index', right_on = 'Speech_index')
SubSpeech.tail()

,Subspeech_index,SubContent,Speech_index,Candidate,Title,Date,Content,Type,URL,Year,Month,Day
198,Speech2762_60:70,[The late panic disclosed a lack of elasticity...,Speech2762,William Howard Taft,\nAddress Accepting the Republican Presidentia...,"\nJuly 28, 1908",\nSenator Warner and Gentlemen of the Committe...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1908,July,28
199,Speech2762_70:80,[The proposition of the Democratic platform is...,Speech2762,William Howard Taft,\nAddress Accepting the Republican Presidentia...,"\nJuly 28, 1908",\nSenator Warner and Gentlemen of the Committe...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1908,July,28
200,Speech2762_80:99,[I have long been of the opinion that the vari...,Speech2762,William Howard Taft,\nAddress Accepting the Republican Presidentia...,"\nJuly 28, 1908",\nSenator Warner and Gentlemen of the Committe...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1908,July,28
201,Speech2763_0:10,"[, Senator Lodge and Gentlemen of the Notifica...",Speech2763,William McKinley,\nAddress Accepting the Republican Presidentia...,"\nJuly 12, 1900",\nSenator Lodge and Gentlemen of the Notificat...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1900,July,12
202,Speech2763_10:28,"[The National credit, which has thus far fortu...",Speech2763,William McKinley,\nAddress Accepting the Republican Presidentia...,"\nJuly 12, 1900",\nSenator Lodge and Gentlemen of the Notificat...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1900,July,12


In [20]:
Sub_Speech_old = pd.read_csv('SubSpeech.csv')

In [21]:
Sub_Speech_old.drop(columns=['Unnamed: 0'], inplace=True)

In [22]:
Sub_Speech_new = Sub_Speech_old.append(SubSpeech)

In [23]:
Sub_Speech_new.tail()

,Subspeech_index,SubContent,Speech_index,Candidate,Title,Date,Content,Type,URL,Year,Month,Day
198,Speech2762_60:70,[The late panic disclosed a lack of elasticity...,Speech2762,William Howard Taft,\nAddress Accepting the Republican Presidentia...,"\nJuly 28, 1908",\nSenator Warner and Gentlemen of the Committe...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1908,July,28
199,Speech2762_70:80,[The proposition of the Democratic platform is...,Speech2762,William Howard Taft,\nAddress Accepting the Republican Presidentia...,"\nJuly 28, 1908",\nSenator Warner and Gentlemen of the Committe...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1908,July,28
200,Speech2762_80:99,[I have long been of the opinion that the vari...,Speech2762,William Howard Taft,\nAddress Accepting the Republican Presidentia...,"\nJuly 28, 1908",\nSenator Warner and Gentlemen of the Committe...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1908,July,28
201,Speech2763_0:10,"[, Senator Lodge and Gentlemen of the Notifica...",Speech2763,William McKinley,\nAddress Accepting the Republican Presidentia...,"\nJuly 12, 1900",\nSenator Lodge and Gentlemen of the Notificat...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1900,July,12
202,Speech2763_10:28,"[The National credit, which has thus far fortu...",Speech2763,William McKinley,\nAddress Accepting the Republican Presidentia...,"\nJuly 12, 1900",\nSenator Lodge and Gentlemen of the Notificat...,Convention Speeches,/documents/address-accepting-the-republican-pr...,1900,July,12


In [24]:
Sub_Speech_new.to_csv('SubSpeech2.csv', index=False)

## Add Penn data

In [25]:
with open('apda.json') as penn_json:
    PennSpeech_dict = json.load(penn_json)

In [133]:
#PennSpeech_dict.keys()

In [26]:
PennSpeech = pd.DataFrame.from_dict(PennSpeech_dict, orient='index')

In [359]:
#PennSpeech.head()

In [27]:
PennSpeech.Date.replace([-999], np.nan, inplace = True)

In [28]:
PennSpeech = PennSpeech[PennSpeech.Genre == 'SPEECH']

In [29]:
Month = []
Day = []

for x in PennSpeech.Date:
    if type(x) == str:
        Month.append(int(re.split('/',x)[0]))
        Day.append(int(re.split('/',x)[-1]))            
    else:
        Month.append(x)
        Day.append(x)

In [30]:
PennSpeech['Month'] = Month
PennSpeech['Day'] = Day

In [31]:
PennSpeech.rename(columns = {'Type':'Chanel', 'Genre':'Type', 'Text':'Content'}, inplace=True)

In [32]:
#Add speech index
PennSpeech['Speech_index'] = ['Speech'+ str(x) for x in range(len(Speech.Year), len(Speech.Year) + len(PennSpeech.Year))]


In [33]:
PennSpeech.drop(columns=['Party', 'Length'], inplace=True)

In [34]:
PennSpeech['Title'] = np.nan
PennSpeech['URL'] = np.nan

In [35]:
##recode candidate names 
PennSpeech.Candidate.replace(['STEVENSON'],'Adlai Stevenson', inplace=True)
PennSpeech.Candidate.replace(['KENNEDY'],'John F. Kennedy', inplace=True)
PennSpeech.Candidate.replace(['NIXON'],'Richard Nixon', inplace=True)
PennSpeech.Candidate.replace(['JOHNSON'],'Lyndon B. Johnson', inplace=True)
PennSpeech.Candidate.replace(['HUMPHREY'],'Hubert H. Humphrey', inplace=True)

PennSpeech.Candidate.replace(['MCGOVERN'],'George McGovern', inplace=True)
PennSpeech.Candidate.replace(['CARTER'],'Jimmy Carter', inplace=True)
PennSpeech.Candidate.replace(['FORD'],'Gerald R. Ford', inplace=True)
PennSpeech.Candidate.replace(['REAGAN'],'Ronald Reagan', inplace=True)
PennSpeech.Candidate.replace(['EISENHOWER'],'Dwight Eisenhower', inplace=True)

PennSpeech.Candidate.replace(['MONDALE'],'Walter F. Mondale', inplace=True)
PennSpeech.Candidate.replace(['DUKAKIS'],'Michael S. Dukakis', inplace=True)
PennSpeech.Candidate.replace(['BUSH'],'George Bush', inplace=True)
PennSpeech.Candidate.replace(['CLINTON'],'William J. Clinton', inplace=True)
PennSpeech.Candidate.replace(['DOLE'],'Robert Dole', inplace=True)

In [36]:
Speech['City'] = np.nan
Speech['State'] = np.nan
Speech['Description'] = np.nan
Speech['Subject'] = np.nan

In [37]:
Speech_merged = Speech.append(PennSpeech)

//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [41]:
Speech_merged.tail()

,Candidate,Chanel,City,Content,Date,Day,Description,Month,Speech_index,State,Subject,Title,Type,URL,Year
995,John F. Kennedy,STUMP,PHOENIX,"Ladies and gentlemen, I just hate to think--wo...",11/03,3.0,AIRPORT,11,Speech5234,ARIZONA,-999,NaN,SPEECH,NaN,1960
996,John F. Kennedy,STUMP,PHOENIX,"My friend and colleague in the House, whom I a...",11/03,3.0,WESTWARD,11,Speech5235,ARIZONA,-999,NaN,SPEECH,NaN,1960
997,John F. Kennedy,STUMP,PHOENIX,"Congressman Udall, my friend and colleague fro...",11/03,3.0,STREET,11,Speech5236,ARIZONA,NATURAL,NaN,SPEECH,NaN,1960
998,John F. Kennedy,STUMP,ALBUQUERQUE,"Mr. Chairman, Governor Burroughs, Senator Chav...",11/03,3.0,-999,11,Speech5237,NEW,"CONGRESS,",NaN,SPEECH,NaN,1960
999,John F. Kennedy,STUMP,AMARILLO,Lyndon; Ralph Yarborough; Mr. Mayor; Senator A...,11/03,3.0,AIRPORT,11,Speech5238,TEXAS,"ECONOMY--GENERAL,",NaN,SPEECH,NaN,1960


In [38]:
PennSpeech.Candidate.value_counts()

John F. Kennedy       312
Dwight Eisenhower     273
Adlai Stevenson       261
Richard Nixon         253
William J. Clinton    187
Ronald Reagan         178
George Bush           167
Hubert H. Humphrey    153
Jimmy Carter          144
Gerald R. Ford        128
Lyndon B. Johnson     111
George McGovern        98
Robert Dole            78
Michael S. Dukakis     66
Walter F. Mondale      66
Name: Candidate, dtype: int64

In [521]:
PennSpeech.to_csv("PennSpeechCleaned.csv", index = False)

In [522]:
Speech_merged.to_csv("CampaignSpeechMerged.csv", index=False)

### create subspeech and merge 

In [14]:
PennSpeech = pd.read_csv('PennSpeechCleaned.csv')

In [371]:
#PennSpeech.head()

In [43]:
sub_dict = {'Subspeech_index':[], 'SubContent':[], 'Speech_index':[]}

In [44]:
# Save every 10 paragraphs into a subspeech e.g. [0,8)

for i,text in enumerate(PennSpeech['Content']): 
    splited = re.split('\n\n',text)
    length = len(re.split('\n\n',text))
    if length > 10:
        a = list(range(0,length,10)) 
        for j in a[:-2]:
            rowvalue = []
            upper=j+10
            sub_dict['Subspeech_index'].append(PennSpeech['Speech_index'][i]+'_'+str(j)+ ':' + str(upper))
            sub_dict['SubContent'].append(splited[j: upper])
            sub_dict['Speech_index'].append(PennSpeech['Speech_index'][i])
            
        sub_dict['Subspeech_index'].append(PennSpeech['Speech_index'][i]+'_'+ str(a[-2])+ ':' + str(length))
        sub_dict['SubContent'].append(splited[a[-2]: length])
        sub_dict['Speech_index'].append(PennSpeech['Speech_index'][i])
    else:
        sub_dict['Subspeech_index'].append(PennSpeech['Speech_index'][i] +'_'+str(0)+ ':' + str(length))
        sub_dict['SubContent'].append(splited[: length])
        sub_dict['Speech_index'].append(PennSpeech['Speech_index'][i])

In [45]:
Sub_Speech = pd.DataFrame.from_dict(sub_dict)
Sub_Speech.head()

,Subspeech_index,SubContent,Speech_index
0,Speech2764_0:10,"[Mr. Tepe, Mr. DiSalle, and distinguished gues...",Speech2764
1,Speech2764_10:20,"[\tThis is my first campaign visit to Ohio, an...",Speech2764
2,Speech2764_20:30,[\tThere is a good reason why voters are suspi...,Speech2764
3,Speech2764_30:48,"[\tWell, we know which way Warren Harding comp...",Speech2764
4,Speech2765_0:16,[Lyndon; Ralph--Senator Yarborough; Mrs. Johns...,Speech2765


In [46]:
SubSpeech_Penn = Sub_Speech.merge(PennSpeech, left_on='Speech_index', right_on = 'Speech_index')

In [424]:
#SubSpeech_Penn.tail()

In [411]:
#Sub_Speech.Speech_index.value_counts()

In [47]:
Sub_Speech2 = pd.read_csv('SubSpeech2.csv')

In [48]:
Sub_Speech2['City'] = np.nan
Sub_Speech2['State'] = np.nan
Sub_Speech2['Description'] = np.nan
Sub_Speech2['Subject'] = np.nan

In [49]:
SubSpeech_Merged = Sub_Speech2.append(SubSpeech_Penn)

In [50]:
SubSpeech_Merged.to_csv('SubSpeech_Merged.csv', index=False)

## Add Annotation Delete possible duplicats

In [3]:
with open('Annotations/test_annotation.jsonl', 'r') as annotation_jsonl:
    annotation_list = list(annotation_jsonl)
with open('Annotations/pop_code1117.jsonl', 'r') as annotation_jsonl:
    annotation_list += list(annotation_jsonl)
with open('Annotations/blind_test1117.jsonl', 'r') as annotation_jsonl:
    annotation_list += list(annotation_jsonl)
with open('Annotations/blindtest2.jsonl', 'r') as annotation_jsonl:
    annotation_list += list(annotation_jsonl)
with open('Annotations/sample1226.jsonl', 'r') as annotation_jsonl:
    annotation_list += list(annotation_jsonl)

In [4]:
len(annotation_list)

435

In [5]:
coded = {}
for annotation in annotation_list:
    annotation_dict = json.loads(annotation)
    coded[annotation_dict['text'].split('##')[0]] = annotation_dict['answer']

In [6]:
coded_data = pd.DataFrame.from_dict(coded, orient='index')
coded_data = coded_data.rename(columns={0:'Pop'})

In [7]:
coded_data['Subspeech_index'] = coded_data.index

In [8]:
Sub_Speech = pd.read_csv('SubSpeech_Merged.csv')

In [10]:
Sub_Speech.tail()

,Candidate,Chanel,City,Content,Date,Day,Description,Month,Speech_index,State,SubContent,Subject,Subspeech_index,Title,Type,URL,Year
19105,John F. Kennedy,STUMP,PHOENIX,"Ladies and gentlemen, I just hate to think--wo...",11/03,3.0,AIRPORT,11.0,Speech5234,ARIZONA,"['Ladies and gentlemen, I just hate to think--...",-999,Speech5234_0:8,NaN,SPEECH,NaN,1960
19106,John F. Kennedy,STUMP,PHOENIX,"My friend and colleague in the House, whom I a...",11/03,3.0,WESTWARD,11.0,Speech5235,ARIZONA,"[""My friend and colleague in the House, whom I...",-999,Speech5235_0:5,NaN,SPEECH,NaN,1960
19107,John F. Kennedy,STUMP,PHOENIX,"Congressman Udall, my friend and colleague fro...",11/03,3.0,STREET,11.0,Speech5236,ARIZONA,"['Congressman Udall, my friend and colleague f...",NATURAL,Speech5236_0:18,NaN,SPEECH,NaN,1960
19108,John F. Kennedy,STUMP,ALBUQUERQUE,"Mr. Chairman, Governor Burroughs, Senator Chav...",11/03,3.0,-999,11.0,Speech5237,NEW,"['Mr. Chairman, Governor Burroughs, Senator Ch...","CONGRESS,",Speech5237_0:17,NaN,SPEECH,NaN,1960
19109,John F. Kennedy,STUMP,AMARILLO,Lyndon; Ralph Yarborough; Mr. Mayor; Senator A...,11/03,3.0,AIRPORT,11.0,Speech5238,TEXAS,['Lyndon; Ralph Yarborough; Mr. Mayor; Senator...,"ECONOMY--GENERAL,",Speech5238_0:15,NaN,SPEECH,NaN,1960


In [11]:
SubSpeech_coded = Sub_Speech.merge(coded_data, how = 'outer', left_on='Subspeech_index', right_on = 'Subspeech_index')

In [12]:
len(SubSpeech_coded.Subspeech_index)

19110

In [13]:
overmonth = ['September', 'October', 'November']

In [15]:
dup_index = SubSpeech_coded[SubSpeech_coded.Year.isin(PennSpeech.Year) & SubSpeech_coded.Type.isin(['Campaign Documents']) 
                            & SubSpeech_coded.Month.isin(overmonth)].index

In [17]:
len(dup_index)

2418

In [18]:
coded_index = SubSpeech_coded[SubSpeech_coded.Year.isin(PennSpeech.Year) & 
                SubSpeech_coded.Month.isin(overmonth) & SubSpeech_coded.Type.isin(['Campaign Documents']) 
                              & SubSpeech_coded.Pop.isin(['reject','accept'])].index

In [19]:
del_index = list(set(dup_index).difference(set(coded_index)))

In [20]:
len(del_index)

2381

In [22]:
SubSpeech_coded_cleaned = SubSpeech_coded.drop(del_index)

In [23]:
len(SubSpeech_coded_cleaned.Year)

16729

In [24]:
SubSpeech_all_cleaned = SubSpeech_coded_cleaned.drop(columns=["Pop"])

In [26]:
len(SubSpeech_all_cleaned.Year)

16729

In [29]:
len(SubSpeech_all_cleaned.loc[SubSpeech_all_cleaned.Year>=1952].Speech_index.unique())

4325

In [35]:
len(SubSpeech_all_cleaned.loc[SubSpeech_all_cleaned.Year.isin(list(range(1952,2016,1)))].Speech_index.unique())

4108

In [30]:
SubSpeech_coded.to_csv("SubSpeech_mergedcoded.csv", index=False)

In [31]:
SubSpeech_coded_cleaned.to_csv('SubSpeech_coded_cleaned.csv', index = False)

In [32]:
SubSpeech_all_cleaned.to_csv("SubSpeech_all_cleaned.csv", index = False)

In [38]:
len(SubSpeech_all_cleaned.Speech_index.unique())

4364

In [39]:
Speech_merged = pd.read_csv('CampaignSpeechMerged.csv')

In [40]:
Speech_merged_cleaned = Speech_merged[Speech_merged.Speech_index.isin(SubSpeech_coded_cleaned.Speech_index)]

In [41]:
len(Speech_merged.Speech_index), len(Speech_merged_cleaned.Speech_index)

(5239, 4364)

In [42]:
Speech_merged_cleaned.to_csv('CampaignSpeechMerged_cleaned.csv', index=False)